In [1]:
import sys
!{sys.executable} -m pip install pyspark delta-spark

     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
      -------------------------------------- 7.6/455.4 MB 41.5 MB/s eta 0:00:11
     - ------------------------------------ 18.9/455.4 MB 48.9 MB/s eta 0:00:09
     -- ----------------------------------- 30.9/455.4 MB 51.7 MB/s eta 0:00:09
     --- ---------------------------------- 43.3/455.4 MB 53.0 MB/s eta 0:00:08
     ---- --------------------------------- 56.1/455.4 MB 55.1 MB/s eta 0:00:08
     ----- -------------------------------- 66.8/455.4 MB 53.8 MB/s eta 0:00:08
     ------ ------------------------------- 79.2/455.4 MB 54.3 MB/s eta 0:00:07
     ------- ------------------------------ 91.8/455.4 MB 55.2 MB/s eta 0:00:07
     -------- ---------------------------- 105.1/455.4 MB 55.9 MB/s eta 0:00:07
     --------- --------------------------- 117.2/455.4 MB 56.0 MB/s eta 0:00:07
     ---------- -------------------------- 129.0/455.4 MB 56.2 MB/s eta 0:00:06
     ----------- ------------------------- 141.

  DEPRECATION: Building 'pyspark' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyspark'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
# Core imports (no Java required!)
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os
import shutil
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")
print("\n💡 Using Pandas to simulate Medallion Architecture")
print("   (Same concepts as PySpark/Databricks, portable without Java)")

✅ Libraries imported successfully!
📦 Pandas version: 2.2.3
📦 NumPy version: 2.1.3

💡 Using Pandas to simulate Medallion Architecture
   (Same concepts as PySpark/Databricks, portable without Java)


In [3]:
# Generate sample raw transaction data (simulating messy source data)

def generate_raw_transactions(num_records=1000):
    """Generate messy raw transaction data"""
    
    random.seed(42)
    np.random.seed(42)
    
    customers = ['CUST001', 'cust002', 'CUST-003', 'cust_004', 'CUST005',
                 'CUST006', 'cust007', 'CUST-008', 'cust_009', 'CUST010']
    
    products = ['Widget A', 'WIDGET A', 'widget a', 'Widget B', 'WIDGET B',
                'Gadget X', 'GADGET X', 'Gadget Y', 'gadget y', 'Tool Z']
    
    regions = ['Northeast', 'NORTHEAST', 'northeast', 'Southeast', 'SOUTHEAST',
               'Midwest', 'MIDWEST', 'Southwest', 'SOUTHWEST', 'West', 'WEST']
    
    statuses = ['completed', 'COMPLETED', 'Completed', 'pending', 'PENDING',
                'cancelled', 'CANCELLED', 'Cancelled']
    
    records = []
    base_date = datetime(2024, 1, 1)
    
    for i in range(num_records):
        days_offset = random.randint(0, 364)
        trans_date = base_date + timedelta(days=days_offset)
        
        date_formats = [
            trans_date.strftime('%Y-%m-%d'),
            trans_date.strftime('%m/%d/%Y'),
            trans_date.strftime('%d-%m-%Y'),
            trans_date.strftime('%Y/%m/%d'),
        ]
        
        record = {
            'transaction_id': f'TXN{str(i+1).zfill(6)}',
            'customer_id': random.choice(customers),
            'product_name': random.choice(products),
            'quantity': random.randint(1, 100),
            'unit_price': round(random.uniform(10.0, 500.0), 2),
            'transaction_date': random.choice(date_formats),
            'region': random.choice(regions),
            'status': random.choice(statuses),
            'discount_pct': random.choice([0, 5, 10, 15, 20, None]),
            'notes': random.choice(['', 'Rush order', 'PRIORITY', 'standard', None])
        }
        records.append(record)
    
    return pd.DataFrame(records)

raw_df = generate_raw_transactions(1000)

print("✅ Generated 1000 raw transaction records")
print(f"\n📊 Sample Raw Data (first 5 rows):")
display(raw_df.head())

print(f"\n📋 Data Types:")
print(raw_df.dtypes)

✅ Generated 1000 raw transaction records

📊 Sample Raw Data (first 5 rows):


,transaction_id,customer_id,product_name,quantity,unit_price,transaction_date,region,status,discount_pct,notes
0,TXN000001,cust002,Widget A,95,144.76,11/23/2024,northeast,COMPLETED,NaN,None
1,TXN000002,CUST010,GADGET X,5,24.60,02/14/2024,Southeast,completed,20.0,Rush order
2,TXN000003,cust_009,GADGET X,29,230.11,28-11-2024,Northeast,Completed,NaN,standard
3,TXN000004,CUST005,widget a,28,479.03,23-06-2024,NORTHEAST,COMPLETED,15.0,
4,TXN000005,CUST006,Tool Z,34,405.49,2024/07/02,SOUTHWEST,COMPLETED,15.0,



📋 Data Types:
transaction_id       object
customer_id          object
product_name         object
quantity              int64
unit_price          float64
transaction_date     object
region               object
status               object
discount_pct        float64
notes                object
dtype: object


In [4]:
# Setup Medallion Architecture Directory Structure

base_path = "./medallion_lakehouse"

bronze_path = f"{base_path}/bronze/transactions"
silver_path = f"{base_path}/silver/transactions"
gold_path = f"{base_path}/gold"

if os.path.exists(base_path):
    shutil.rmtree(base_path)

for path in [bronze_path, silver_path, f"{gold_path}/daily_summary", f"{gold_path}/customer_metrics", f"{gold_path}/product_metrics"]:
    os.makedirs(path, exist_ok=True)

print("✅ Medallion Lakehouse Directory Structure Created!")
print(f"""
📁 {base_path}/
   ├── 🥉 bronze/
   │      └── transactions/     (Raw ingested data)
   ├── 🥈 silver/
   │      └── transactions/     (Cleaned & standardized)
   └── 🥇 gold/
          ├── daily_summary/    (Daily aggregations)
          ├── customer_metrics/ (Customer analytics)
          └── product_metrics/  (Product analytics)
""")

✅ Medallion Lakehouse Directory Structure Created!

📁 ./medallion_lakehouse/
   ├── 🥉 bronze/
   │      └── transactions/     (Raw ingested data)
   ├── 🥈 silver/
   │      └── transactions/     (Cleaned & standardized)
   └── 🥇 gold/
          ├── daily_summary/    (Daily aggregations)
          ├── customer_metrics/ (Customer analytics)
          └── product_metrics/  (Product analytics)



In [5]:
# ============================================================
# 🥉 BRONZE LAYER - Raw Data Ingestion
# ============================================================

print("=" * 60)
print("🥉 BRONZE LAYER - Raw Data Ingestion")
print("=" * 60)

bronze_df = raw_df.copy()

bronze_df['_ingestion_timestamp'] = datetime.now()
bronze_df['_source_system'] = 'transaction_system'
bronze_df['_file_name'] = 'raw_transactions_2024.csv'

print("\n📋 Bronze Layer Schema:")
print(bronze_df.dtypes)

print("\n📊 Bronze Layer Sample Data:")
display(bronze_df.head())

bronze_df.to_parquet(f"{bronze_path}/transactions.parquet", index=False)

bronze_count = len(bronze_df)
print(f"\n✅ Bronze Layer Complete!")
print(f"📊 Records ingested: {bronze_count}")
print(f"📁 Saved to: {bronze_path}/transactions.parquet")

🥉 BRONZE LAYER - Raw Data Ingestion

📋 Bronze Layer Schema:
transaction_id                  object
customer_id                     object
product_name                    object
quantity                         int64
unit_price                     float64
transaction_date                object
region                          object
status                          object
discount_pct                   float64
notes                           object
_ingestion_timestamp    datetime64[us]
_source_system                  object
_file_name                      object
dtype: object

📊 Bronze Layer Sample Data:


,transaction_id,customer_id,product_name,quantity,unit_price,transaction_date,region,status,discount_pct,notes,_ingestion_timestamp,_source_system,_file_name
0,TXN000001,cust002,Widget A,95,144.76,11/23/2024,northeast,COMPLETED,NaN,None,2026-01-22 22:31:24.173827,transaction_system,raw_transactions_2024.csv
1,TXN000002,CUST010,GADGET X,5,24.60,02/14/2024,Southeast,completed,20.0,Rush order,2026-01-22 22:31:24.173827,transaction_system,raw_transactions_2024.csv
2,TXN000003,cust_009,GADGET X,29,230.11,28-11-2024,Northeast,Completed,NaN,standard,2026-01-22 22:31:24.173827,transaction_system,raw_transactions_2024.csv
3,TXN000004,CUST005,widget a,28,479.03,23-06-2024,NORTHEAST,COMPLETED,15.0,,2026-01-22 22:31:24.173827,transaction_system,raw_transactions_2024.csv
4,TXN000005,CUST006,Tool Z,34,405.49,2024/07/02,SOUTHWEST,COMPLETED,15.0,,2026-01-22 22:31:24.173827,transaction_system,raw_transactions_2024.csv



✅ Bronze Layer Complete!
📊 Records ingested: 1000
📁 Saved to: ./medallion_lakehouse/bronze/transactions/transactions.parquet


In [6]:
# ============================================================
# 🥈 SILVER LAYER - Data Cleaning & Standardization
# ============================================================

print("=" * 60)
print("🥈 SILVER LAYER - Data Cleaning & Standardization")
print("=" * 60)

bronze_raw = pd.read_parquet(f"{bronze_path}/transactions.parquet")

silver_df = bronze_raw.copy()

# 1. Standardize customer_id (uppercase, remove special characters)
import re
silver_df['customer_id'] = silver_df['customer_id'].apply(
    lambda x: re.sub(r'[^A-Za-z0-9]', '', str(x)).upper()
)

# 2. Standardize product_name (title case)
silver_df['product_name'] = silver_df['product_name'].str.title()

# 3. Standardize region (title case)
silver_df['region'] = silver_df['region'].str.title()

# 4. Standardize status (lowercase)
silver_df['status'] = silver_df['status'].str.lower()

# 5. Parse and standardize dates
def parse_date(date_str):
    formats = ['%Y-%m-%d', '%m/%d/%Y', '%d-%m-%Y', '%Y/%m/%d']
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    return pd.to_datetime(date_str)

silver_df['transaction_date'] = silver_df['transaction_date'].apply(parse_date)

# 6. Handle null discount_pct (default to 0)
silver_df['discount_pct'] = silver_df['discount_pct'].fillna(0)

# 7. Calculate derived columns
silver_df['gross_amount'] = silver_df['quantity'] * silver_df['unit_price']
silver_df['discount_amount'] = silver_df['gross_amount'] * (silver_df['discount_pct'] / 100)
silver_df['net_amount'] = silver_df['gross_amount'] - silver_df['discount_amount']

# 8. Add date parts for partitioning/filtering
silver_df['year'] = silver_df['transaction_date'].dt.year
silver_df['month'] = silver_df['transaction_date'].dt.month
silver_df['day'] = silver_df['transaction_date'].dt.day

# 9. Add processing metadata
silver_df['_processing_timestamp'] = datetime.now()

print("\n📋 Silver Layer Schema:")
print(silver_df.dtypes)

print("\n📊 Silver Layer Sample Data (cleaned):")
display(silver_df[['transaction_id', 'customer_id', 'product_name', 
                   'quantity', 'unit_price', 'transaction_date',
                   'region', 'status', 'net_amount']].head(10))

silver_df.to_parquet(f"{silver_path}/transactions.parquet", index=False)

silver_count = len(silver_df)
print(f"\n✅ Silver Layer Complete!")
print(f"📊 Records processed: {silver_count}")
print(f"📁 Saved to: {silver_path}/transactions.parquet")

🥈 SILVER LAYER - Data Cleaning & Standardization

📋 Silver Layer Schema:
transaction_id                   object
customer_id                      object
product_name                     object
quantity                          int64
unit_price                      float64
transaction_date         datetime64[ns]
region                           object
status                           object
discount_pct                    float64
notes                            object
_ingestion_timestamp     datetime64[us]
_source_system                   object
_file_name                       object
gross_amount                    float64
discount_amount                 float64
net_amount                      float64
year                              int32
month                             int32
day                               int32
_processing_timestamp    datetime64[us]
dtype: object

📊 Silver Layer Sample Data (cleaned):


,transaction_id,customer_id,product_name,quantity,unit_price,transaction_date,region,status,net_amount
0,TXN000001,CUST002,Widget A,95,144.76,2024-11-23,Northeast,completed,13752.200
1,TXN000002,CUST010,Gadget X,5,24.60,2024-02-14,Southeast,completed,98.400
2,TXN000003,CUST009,Gadget X,29,230.11,2024-11-28,Northeast,completed,6673.190
3,TXN000004,CUST005,Widget A,28,479.03,2024-06-23,Northeast,completed,11400.914
4,TXN000005,CUST006,Tool Z,34,405.49,2024-07-02,Southwest,completed,11718.661
5,TXN000006,CUST005,Tool Z,47,292.90,2024-10-09,Northeast,pending,12389.670
6,TXN000007,CUST002,Gadget X,36,232.17,2024-04-29,Northeast,cancelled,7522.308
7,TXN000008,CUST005,Widget A,78,321.14,2024-12-09,Northeast,cancelled,21291.582
8,TXN000009,CUST009,Widget B,88,168.90,2024-11-23,Southeast,completed,13376.880
9,TXN000010,CUST002,Widget B,73,439.42,2024-05-17,Southeast,cancelled,27266.011



✅ Silver Layer Complete!
📊 Records processed: 1000
📁 Saved to: ./medallion_lakehouse/silver/transactions/transactions.parquet


In [7]:
# ============================================================
# 📊 DATA QUALITY CHECKS - Silver Layer Validation
# ============================================================

print("=" * 60)
print("📊 DATA QUALITY CHECKS")
print("=" * 60)

silver_data = pd.read_parquet(f"{silver_path}/transactions.parquet")

print("\n1️⃣ NULL VALUE CHECK:")
print("-" * 40)
null_counts = silver_data.isnull().sum()
for col, count in null_counts.items():
    if count > 0:
        pct = (count / len(silver_data)) * 100
        print(f"   ⚠️ {col}: {count} nulls ({pct:.2f}%)")
if null_counts.sum() == 0:
    print("   ✅ No null values found!")

print("\n2️⃣ UNIQUE VALUE CHECK:")
print("-" * 40)
print(f"   • Unique customers: {silver_data['customer_id'].nunique()}")
print(f"   • Unique products: {silver_data['product_name'].nunique()}")
print(f"   • Unique regions: {silver_data['region'].nunique()}")
print(f"   • Unique statuses: {silver_data['status'].nunique()}")

print("\n3️⃣ DATE RANGE CHECK:")
print("-" * 40)
print(f"   • Date range: {silver_data['transaction_date'].min()} to {silver_data['transaction_date'].max()}")

print("\n4️⃣ NUMERIC RANGE CHECK:")
print("-" * 40)
print(f"   • Net amount range: ${silver_data['net_amount'].min():.2f} to ${silver_data['net_amount'].max():.2f}")
print(f"   • Average net amount: ${silver_data['net_amount'].mean():.2f}")

print("\n5️⃣ STANDARDIZATION CHECK:")
print("-" * 40)
print("   Unique regions after standardization:")
print(f"   {silver_data['region'].unique().tolist()}")
print("\n   Unique statuses after standardization:")
print(f"   {silver_data['status'].unique().tolist()}")

print("\n✅ Data Quality Checks Complete!")


📊 DATA QUALITY CHECKS

1️⃣ NULL VALUE CHECK:
----------------------------------------
   ⚠️ notes: 203 nulls (20.30%)

2️⃣ UNIQUE VALUE CHECK:
----------------------------------------
   • Unique customers: 10
   • Unique products: 5
   • Unique regions: 5
   • Unique statuses: 3

3️⃣ DATE RANGE CHECK:
----------------------------------------
   • Date range: 2024-01-01 00:00:00 to 2024-12-30 00:00:00

4️⃣ NUMERIC RANGE CHECK:
----------------------------------------
   • Net amount range: $13.85 to $48808.90
   • Average net amount: $12481.21

5️⃣ STANDARDIZATION CHECK:
----------------------------------------
   Unique regions after standardization:
   ['Northeast', 'Southeast', 'Southwest', 'West', 'Midwest']

   Unique statuses after standardization:
   ['completed', 'pending', 'cancelled']

✅ Data Quality Checks Complete!


In [8]:
# ============================================================
# 🥇 GOLD LAYER - Business Aggregations
# ============================================================

print("=" * 60)
print("🥇 GOLD LAYER - Business Aggregations")
print("=" * 60)

silver_data = pd.read_parquet(f"{silver_path}/transactions.parquet")

# ============================================================
# GOLD TABLE 1: Daily Sales Summary
# ============================================================
print("\n📊 Creating Gold Table: Daily Sales Summary")
print("-" * 40)

completed_data = silver_data[silver_data['status'] == 'completed']

daily_summary = completed_data.groupby(['transaction_date', 'region']).agg(
    total_transactions=('transaction_id', 'count'),
    total_units=('quantity', 'sum'),
    gross_revenue=('gross_amount', 'sum'),
    total_discounts=('discount_amount', 'sum'),
    net_revenue=('net_amount', 'sum'),
    avg_order_value=('net_amount', 'mean')
).reset_index()

daily_summary['_aggregation_timestamp'] = datetime.now()
daily_summary = daily_summary.sort_values(['transaction_date', 'region'])

print("\n📈 Daily Sales Summary (sample):")
display(daily_summary.head(10))

daily_summary.to_parquet(f"{gold_path}/daily_summary/summary.parquet", index=False)
print(f"✅ Saved to: {gold_path}/daily_summary/summary.parquet")

🥇 GOLD LAYER - Business Aggregations

📊 Creating Gold Table: Daily Sales Summary
----------------------------------------

📈 Daily Sales Summary (sample):


,transaction_date,region,total_transactions,total_units,gross_revenue,total_discounts,net_revenue,avg_order_value,_aggregation_timestamp
0,2024-01-03,Midwest,1,53,14964.02,0.000,14964.020,14964.020,2026-01-22 22:32:03.595716
1,2024-01-05,Midwest,1,89,21616.32,1080.816,20535.504,20535.504,2026-01-22 22:32:03.595716
2,2024-01-07,Northeast,1,74,24998.68,0.000,24998.680,24998.680,2026-01-22 22:32:03.595716
3,2024-01-09,Southeast,1,57,13320.33,0.000,13320.330,13320.330,2026-01-22 22:32:03.595716
4,2024-01-10,Southwest,1,58,11446.88,572.344,10874.536,10874.536,2026-01-22 22:32:03.595716
5,2024-01-11,West,1,50,19597.50,0.000,19597.500,19597.500,2026-01-22 22:32:03.595716
6,2024-01-14,Midwest,1,51,13054.98,1305.498,11749.482,11749.482,2026-01-22 22:32:03.595716
7,2024-01-14,Southeast,1,98,48808.90,0.000,48808.900,48808.900,2026-01-22 22:32:03.595716
8,2024-01-15,Midwest,1,60,14870.40,743.520,14126.880,14126.880,2026-01-22 22:32:03.595716
9,2024-01-15,Northeast,1,92,30220.16,3022.016,27198.144,27198.144,2026-01-22 22:32:03.595716


✅ Saved to: ./medallion_lakehouse/gold/daily_summary/summary.parquet


In [9]:
# ============================================================
# GOLD TABLE 2: Customer Metrics
# ============================================================
print("\n📊 Creating Gold Table: Customer Metrics")
print("-" * 40)

customer_metrics = completed_data.groupby('customer_id').agg(
    total_orders=('transaction_id', 'count'),
    total_units_purchased=('quantity', 'sum'),
    total_spend=('net_amount', 'sum'),
    avg_order_value=('net_amount', 'mean'),
    first_purchase_date=('transaction_date', 'min'),
    last_purchase_date=('transaction_date', 'max'),
    orders_with_discount=('discount_pct', lambda x: (x > 0).sum())
).reset_index()

customer_metrics['_aggregation_timestamp'] = datetime.now()

customer_metrics['spend_rank'] = customer_metrics['total_spend'].rank(ascending=False, method='dense').astype(int)

customer_metrics = customer_metrics.sort_values('total_spend', ascending=False)

print("\n👥 Customer Metrics (Top 10 by spend):")
display(customer_metrics.head(10))

customer_metrics.to_parquet(f"{gold_path}/customer_metrics/metrics.parquet", index=False)
print(f"✅ Saved to: {gold_path}/customer_metrics/metrics.parquet")


📊 Creating Gold Table: Customer Metrics
----------------------------------------

👥 Customer Metrics (Top 10 by spend):


,customer_id,total_orders,total_units_purchased,total_spend,avg_order_value,first_purchase_date,last_purchase_date,orders_with_discount,_aggregation_timestamp,spend_rank
6,CUST007,41,2433,620597.4740,15136.523756,2024-01-14,2024-12-30,29,2026-01-22 22:32:17.156913,1
1,CUST002,43,2344,525508.8505,12221.136058,2024-01-07,2024-12-29,22,2026-01-22 22:32:17.156913,2
9,CUST010,37,1919,509997.6540,13783.720378,2024-01-10,2024-11-29,27,2026-01-22 22:32:17.156913,3
0,CUST001,43,2124,498107.3305,11583.891407,2024-01-03,2024-12-28,24,2026-01-22 22:32:17.156913,4
7,CUST008,31,1574,436013.4185,14064.948984,2024-02-10,2024-12-17,21,2026-01-22 22:32:17.156913,5
8,CUST009,36,1632,431231.6225,11978.656181,2024-01-27,2024-12-22,27,2026-01-22 22:32:17.156913,6
5,CUST006,32,1747,419126.1095,13097.690922,2024-01-15,2024-12-19,21,2026-01-22 22:32:17.156913,7
2,CUST003,27,1472,401172.2930,14858.233074,2024-01-27,2024-12-21,18,2026-01-22 22:32:17.156913,8
3,CUST004,43,1993,386955.3045,8998.960570,2024-01-11,2024-12-30,24,2026-01-22 22:32:17.156913,9
4,CUST005,30,1549,317075.0050,10569.166833,2024-02-07,2024-12-23,19,2026-01-22 22:32:17.156913,10


✅ Saved to: ./medallion_lakehouse/gold/customer_metrics/metrics.parquet


In [10]:
# ============================================================
# GOLD TABLE 2: Customer Metrics
# ============================================================
print("\n📊 Creating Gold Table: Customer Metrics")
print("-" * 40)

customer_metrics = completed_data.groupby('customer_id').agg(
    total_orders=('transaction_id', 'count'),
    total_units_purchased=('quantity', 'sum'),
    total_spend=('net_amount', 'sum'),
    avg_order_value=('net_amount', 'mean'),
    first_purchase_date=('transaction_date', 'min'),
    last_purchase_date=('transaction_date', 'max'),
    orders_with_discount=('discount_pct', lambda x: (x > 0).sum())
).reset_index()

customer_metrics['_aggregation_timestamp'] = datetime.now()

customer_metrics['spend_rank'] = customer_metrics['total_spend'].rank(ascending=False, method='dense').astype(int)

customer_metrics = customer_metrics.sort_values('total_spend', ascending=False)

print("\n👥 Customer Metrics (Top 10 by spend):")
display(customer_metrics.head(10))

customer_metrics.to_parquet(f"{gold_path}/customer_metrics/metrics.parquet", index=False)
print(f"✅ Saved to: {gold_path}/customer_metrics/metrics.parquet")


📊 Creating Gold Table: Customer Metrics
----------------------------------------

👥 Customer Metrics (Top 10 by spend):


,customer_id,total_orders,total_units_purchased,total_spend,avg_order_value,first_purchase_date,last_purchase_date,orders_with_discount,_aggregation_timestamp,spend_rank
6,CUST007,41,2433,620597.4740,15136.523756,2024-01-14,2024-12-30,29,2026-01-22 22:32:28.103531,1
1,CUST002,43,2344,525508.8505,12221.136058,2024-01-07,2024-12-29,22,2026-01-22 22:32:28.103531,2
9,CUST010,37,1919,509997.6540,13783.720378,2024-01-10,2024-11-29,27,2026-01-22 22:32:28.103531,3
0,CUST001,43,2124,498107.3305,11583.891407,2024-01-03,2024-12-28,24,2026-01-22 22:32:28.103531,4
7,CUST008,31,1574,436013.4185,14064.948984,2024-02-10,2024-12-17,21,2026-01-22 22:32:28.103531,5
8,CUST009,36,1632,431231.6225,11978.656181,2024-01-27,2024-12-22,27,2026-01-22 22:32:28.103531,6
5,CUST006,32,1747,419126.1095,13097.690922,2024-01-15,2024-12-19,21,2026-01-22 22:32:28.103531,7
2,CUST003,27,1472,401172.2930,14858.233074,2024-01-27,2024-12-21,18,2026-01-22 22:32:28.103531,8
3,CUST004,43,1993,386955.3045,8998.960570,2024-01-11,2024-12-30,24,2026-01-22 22:32:28.103531,9
4,CUST005,30,1549,317075.0050,10569.166833,2024-02-07,2024-12-23,19,2026-01-22 22:32:28.103531,10


✅ Saved to: ./medallion_lakehouse/gold/customer_metrics/metrics.parquet


In [12]:
# ============================================================
# GOLD TABLE 3: Product Metrics
# ============================================================
print("\n📊 Creating Gold Table: Product Metrics")
print("-" * 40)

product_metrics = completed_data.groupby('product_name').agg(
    total_orders=('transaction_id', 'count'),
    total_units_sold=('quantity', 'sum'),
    total_revenue=('net_amount', 'sum'),
    avg_unit_price=('unit_price', 'mean'),
    avg_discount_pct=('discount_pct', 'mean'),
    customer_count=('customer_id', 'nunique')
).reset_index()

product_metrics['_aggregation_timestamp'] = datetime.now()

product_metrics['revenue_rank'] = product_metrics['total_revenue'].rank(ascending=False, method='dense').astype(int)

product_metrics = product_metrics.sort_values('total_revenue', ascending=False)

print("\n📦 Product Metrics (by revenue):")
display(product_metrics)

product_metrics.to_parquet(f"{gold_path}/product_metrics/metrics.parquet", index=False)
print(f"✅ Saved to: {gold_path}/product_metrics/metrics.parquet")


📊 Creating Gold Table: Product Metrics
----------------------------------------

📦 Product Metrics (by revenue):


,product_name,total_orders,total_units_sold,total_revenue,avg_unit_price,avg_discount_pct,customer_count,_aggregation_timestamp,revenue_rank
3,Widget A,100,5056,1.223387e+06,256.491800,7.750000,10,2026-01-22 22:33:13.302214,1
0,Gadget X,79,4222,1.019434e+06,244.777215,8.291139,10,2026-01-22 22:33:13.302214,2
4,Widget B,76,4072,9.605894e+05,248.117237,7.828947,10,2026-01-22 22:33:13.302214,3
1,Gadget Y,70,3425,8.592840e+05,258.856857,8.500000,10,2026-01-22 22:33:13.302214,4
2,Tool Z,38,2012,4.830901e+05,262.636579,7.894737,10,2026-01-22 22:33:13.302214,5


✅ Saved to: ./medallion_lakehouse/gold/product_metrics/metrics.parquet


In [13]:
# ============================================================
# 📋 MEDALLION PIPELINE SUMMARY
# ============================================================

print("=" * 60)
print("📋 MEDALLION PIPELINE SUMMARY")
print("=" * 60)

bronze_final = pd.read_parquet(f"{bronze_path}/transactions.parquet")
silver_final = pd.read_parquet(f"{silver_path}/transactions.parquet")
gold_daily = pd.read_parquet(f"{gold_path}/daily_summary/summary.parquet")
gold_customer = pd.read_parquet(f"{gold_path}/customer_metrics/metrics.parquet")
gold_product = pd.read_parquet(f"{gold_path}/product_metrics/metrics.parquet")

print(f"""
🥉 BRONZE LAYER (Raw Data)
   ├── Records: {len(bronze_final)}
   ├── Columns: {len(bronze_final.columns)}
   └── Path: {bronze_path}

🥈 SILVER LAYER (Cleaned Data)
   ├── Records: {len(silver_final)}
   ├── Columns: {len(silver_final.columns)}
   └── Path: {silver_path}

🥇 GOLD LAYER (Business Aggregations)
   ├── Daily Summary
   │      ├── Records: {len(gold_daily)}
   │      └── Path: {gold_path}/daily_summary
   ├── Customer Metrics
   │      ├── Records: {len(gold_customer)}
   │      └── Path: {gold_path}/customer_metrics
   └── Product Metrics
          ├── Records: {len(gold_product)}
          └── Path: {gold_path}/product_metrics
""")

completed_orders = silver_final[silver_final['status'] == 'completed']
total_revenue = completed_orders['net_amount'].sum()

print(f"""
📈 KEY BUSINESS METRICS
   ├── Total Completed Orders: {len(completed_orders)}
   ├── Total Revenue: ${total_revenue:,.2f}
   ├── Unique Customers: {len(gold_customer)}
   └── Unique Products: {len(gold_product)}
""")

print("=" * 60)
print("🎉 MEDALLION PIPELINE COMPLETE!")
print("=" * 60)

📋 MEDALLION PIPELINE SUMMARY

🥉 BRONZE LAYER (Raw Data)
   ├── Records: 1000
   ├── Columns: 13
   └── Path: ./medallion_lakehouse/bronze/transactions

🥈 SILVER LAYER (Cleaned Data)
   ├── Records: 1000
   ├── Columns: 20
   └── Path: ./medallion_lakehouse/silver/transactions

🥇 GOLD LAYER (Business Aggregations)
   ├── Daily Summary
   │      ├── Records: 341
   │      └── Path: ./medallion_lakehouse/gold/daily_summary
   ├── Customer Metrics
   │      ├── Records: 10
   │      └── Path: ./medallion_lakehouse/gold/customer_metrics
   └── Product Metrics
          ├── Records: 5
          └── Path: ./medallion_lakehouse/gold/product_metrics


📈 KEY BUSINESS METRICS
   ├── Total Completed Orders: 363
   ├── Total Revenue: $4,545,785.06
   ├── Unique Customers: 10
   └── Unique Products: 5

🎉 MEDALLION PIPELINE COMPLETE!


In [14]:
# ============================================================
# 💾 EXPORT GOLD TABLES TO CSV
# ============================================================

print("💾 Exporting Gold tables to CSV...")

gold_daily.to_csv("gold_daily_summary.csv", index=False)
gold_customer.to_csv("gold_customer_metrics.csv", index=False)
gold_product.to_csv("gold_product_metrics.csv", index=False)

print("✅ Exported to CSV files:")
print("   • gold_daily_summary.csv")
print("   • gold_customer_metrics.csv")
print("   • gold_product_metrics.csv")

print("""
============================================================
🎉 MEDALLION PIPELINE PROJECT COMPLETE!
============================================================

This pipeline demonstrated:
  ✅ Bronze Layer - Raw data ingestion with metadata
  ✅ Silver Layer - Data cleaning & standardization
  ✅ Gold Layer - Business aggregations
  ✅ Data Quality Checks
  ✅ Parquet file storage (columnar format)
  ✅ Customer & Product analytics

Skills Showcased:
  • Medallion Architecture design (Bronze → Silver → Gold)
  • Data transformation & cleaning
  • Aggregations & window functions
  • Data quality validation
  • Lakehouse patterns
  • Parquet file format

Relevant for:
  • Databricks Engineer roles
  • Data Engineer positions
  • Analytics Engineering
  
Note: This implementation uses Pandas to demonstrate the concepts.
In production, this would run on PySpark/Databricks at scale.
============================================================
""")

💾 Exporting Gold tables to CSV...
✅ Exported to CSV files:
   • gold_daily_summary.csv
   • gold_customer_metrics.csv
   • gold_product_metrics.csv

🎉 MEDALLION PIPELINE PROJECT COMPLETE!

This pipeline demonstrated:
  ✅ Bronze Layer - Raw data ingestion with metadata
  ✅ Silver Layer - Data cleaning & standardization
  ✅ Gold Layer - Business aggregations
  ✅ Data Quality Checks
  ✅ Parquet file storage (columnar format)
  ✅ Customer & Product analytics

Skills Showcased:
  • Medallion Architecture design (Bronze → Silver → Gold)
  • Data transformation & cleaning
  • Aggregations & window functions
  • Data quality validation
  • Lakehouse patterns
  • Parquet file format

Relevant for:
  • Databricks Engineer roles
  • Data Engineer positions
  • Analytics Engineering
  
Note: This implementation uses Pandas to demonstrate the concepts.
In production, this would run on PySpark/Databricks at scale.

